In [ ]:
import PyPDF2
import pandas as pd

# Read pdf object
pdfFileObj = open('20221118-FULL.pdf', 'rb') 
pdfReader = PyPDF2.PdfReader(pdfFileObj) 

allText = ""
for i in range(792): #Iterating over n pages 
    pageObj = pdfReader.pages[i]
    page = pageObj.extract_text()
    allText += page #Store page content into string variable

# Preprocessing
allText = allText.replace(';','')

alltext_string_all = allText.replace('Legal basis',';Legal basis').replace('Identity information',';Identity information').replace('Programme', ';Programme').replace('Birth information',';Birth information').replace('Citizenship information',';Citizenship information').replace('Contact information',';Contact information')
testListSplit = alltext_string_all.split('reference number: EU')


# Create dataframe
df = pd.DataFrame([sub.split(";") for sub in testListSplit])
df.fillna("",inplace=True)
df.columns = ['EU Reference', 'Legal basis', 'Programme', 'Identity information', 'Birth information', 'Citizenship information', 'Contact information']

# Dataframe Cleanup
def cleanupDataframe(dataframe):
    dataframe['Legal basis'] = dataframe['Legal basis'].apply(lambda x: x.replace('Legal basis: ','').replace('\n',''))
    dataframe['Programme'] = dataframe['Programme'].apply(lambda x: x.replace('Programme: ','').replace('\n',''))
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: x.replace('Identity information:','').replace('Name/Alias:',';').replace('•','').replace('Function:','').split(';'))
    dataframe.replace('^\s+', '', regex=True, inplace=True) #front
    dataframe.replace('\s+$', '', regex=True, inplace=True) #end
    dataframe['Birth information'] = dataframe['Birth information'].apply(lambda x: x.replace('Birth information:\n•',''))
    return dataframe

def stripDataframe(dataframe):
    dataframe['Identity information'] = dataframe['Identity information'].apply(lambda x: [y.strip() for y in x ])
    return dataframe

cleanupDataframe(df)
stripDataframe(df)